<a href="https://colab.research.google.com/github/Dinesh9928/Data_science_protfolio/blob/main/StockMarket_Prediction_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance

Loading main data

In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [6]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


In [7]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [8]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [9]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-26,2666.570068,2684.739990,2646.030029,2680.699951,2680.699951,1526300
2021-07-27,2685.010010,2687.979980,2602.080078,2638.000000,2638.000000,2714500
2021-07-28,2726.239014,2765.939941,2706.010010,2742.629883,2742.629883,2555527


In [10]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [11]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [12]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,899.000000,899.000000,899.000000,899.000000,899.000000,8.990000e+02
mean,1414.097139,1429.073935,1399.703039,1415.104626,1415.104626,1.812625e+06
std,400.345467,403.328564,397.948012,401.173130,401.173130,8.420698e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1132.315002,1144.169983,1117.505005,1132.085022,1132.085022,1.274700e+06
50%,1241.989990,1250.000000,1228.540039,1241.130005,1241.130005,1.585000e+06
75%,1524.114990,1546.164978,1508.534973,1522.184998,1522.184998,2.079300e+06
max,2726.239014,2765.939941,2706.010010,2742.629883,2742.629883,6.658900e+06


Understanding Trends with in the Data

In [13]:
import plotly.graph_objects as go

In [14]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [15]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [32]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [33]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


Scrapping extra information

In [34]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [35]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [36]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [37]:
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [38]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']

In [39]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [40]:
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [41]:
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [42]:
data.head()

,Close,Volume,RSI
2018-01-02,1073.209961,1588300,65.4310
2018-01-03,1091.520020,1565900,78.3686
2018-01-04,1095.760010,1302600,80.4830
2018-01-05,1110.290039,1512500,86.2429
2018-01-08,1114.209961,1232200,87.4882


In [43]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [44]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [45]:
X , Y = CreateFeatures_and_Targets(data , 32)

In [46]:
# Check the shapes
X.shape , Y.shape

((866, 32, 3), (866,))

In [47]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [48]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 3), (639,))

In [49]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((227, 32, 3), (227,))

In [50]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [51]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [52]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [53]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [54]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [55]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [57]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [59]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 133s 208ms/step - loss: 0.0076 - val_loss: 0.5101
Epoch 2/10
639/639 [==============================] - 131s 205ms/step - loss: 0.0096 - val_loss: 0.3668
Epoch 3/10
639/639 [==============================] - 131s 206ms/step - loss: 0.0077 - val_loss: 0.2770
Epoch 4/10
639/639 [==============================] - 132s 207ms/step - loss: 0.0069 - val_loss: 0.2409
Epoch 5/10
639/639 [==============================] - 131s 206ms/step - loss: 0.0058 - val_loss: 0.1681
Epoch 6/10
639/639 [==============================] - 132s 207ms/step - loss: 0.0055 - val_loss: 0.1643
Epoch 7/10
639/639 [==============================] - 132s 207ms/step - loss: 0.0044 - val_loss: 0.1567
Epoch 8/10
639/639 [==============================] - 131s 205ms/step - loss: 0.0047 - val_loss: 0.1496
Epoch 9/10
639/639 [==============================] - 131s 204ms/step - loss: 0.0039 - val_loss: 0.1391
Epoch 10/10
639/639 [==============================] - 130s 204m

In [60]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [61]:
Predictions = model.predict(Xtest)

In [62]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [63]:
Predictions.shape

(227, 1)

In [64]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [65]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [66]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [67]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [68]:
Predictions = model.predict(Total_features)

In [69]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [70]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [71]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [72]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [73]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [74]:
PredictStockPrice(loaded_model , data , '2021-01-14')

1688.7292